In [ ]:
# default_exp models.ultragcn

# UltraGCN
> An efficient graph-convolutional recommendation model.

Industrial recommender systems usually involve massive graphs due to the large numbers of users and items. However, current GCN-based models are hard to train with large graphs, which hinders their wide adoption in industry. This brings efficiency and scalability challenges for model designs. Some efforts have been made to simplify the design of GCN-based CF models, mainly by removing feature transformations and non-linear activations that are not necessary for CF. These proved beneficial. But it has been seen that message passing (i.e., neighborhood aggregation) on a large graph is usually time-consuming for CF. In particular, stacking multiple layers of message passing could lead to the slow convergence of GCN-based models on CF tasks. For example, in our experiments, three-layer LightGCN takes more than 700 epochs to converge to its best result on the Amazon Books dataset, which would be unacceptable in an industrial setting.

UltraGCN has the following optimization objective:

$$\mathcal{L} = \mathcal{L}_O + \lambda \mathcal{L}_C$$

where 𝜆 is the hyper-parameter to control the importance weights of two loss terms.

Moreover, except for user-item relationships, some other relationships (e.g., item-item and user-user relationships) also greatly contribute to the effectiveness of GCN-based models on CF. However, in conventional GCN-based models, these relationships are implicitly learned through the same message passing layers with user-item relationships. This not only leads to the unreasonable edge weight assignments, but also fails to capture the relative importances of different types of relationships.

<img src='https://github.com/recohut/reco-static/raw/master/media/images/models/ultragcn.png'>

### Constraint Loss

Instead of performing explicit message passing, LightGCN aims to directly approximate the convergence state by normalizing the embeddings to unit vectors and then maximize the dot product of both terms, which is equivalent to maximize the cosine similarity between $𝑒_u$ and $e_i$.

$$\max \sum_{i \in \mathcal{N}(u)} \beta_{u,i}e_u^Te_i,\ \forall u \in U$$

For ease of optimization, we further incorporate sigmoid activation and negative log likelihood, and derive the following loss:

$$\mathcal{L}_C = - \sum_{u \in U} \sum_{i \in \mathcal{N}(u)} \beta_{u,i} \log(\sigma(e_u^Te_i)),$$

To avoid over-smoothing (users and items could easily converge to the same embeddings), UltraGCN perform negative sampling (inspired by Word2vec). The constraint loss would then become:

$$\mathcal{L}_C = - \sum_{(u,i) \in N^+} \beta_{u,i} \log(\sigma(e_u^Te_i)) - \sum_{(u,j) \in N^-} \beta_{u,j} \log(\sigma(e_u^Te_j))$$

### Optimization Loss

Typically, CF models perform item recommendation by applying either pairwise BPR (Bayesian personalized ranking) loss or pointwise BCE (binary cross-entropy) loss for optimization. UltraGCN formulate CF as a link prediction problem in graph learning, and therefore chooses the following BCE loss as the main optimization objective. It is also consistent with the loss format of $\mathcal{L}_C$:

$$\mathcal{L}_O = - \sum_{(u,i) \in N^+} \log(\sigma(e_u^Te_i)) - \sum_{(u,j) \in N^-} \log(\sigma(e_u^Te_j))$$

### Learning on Item-Item

Moreover, except for user-item relationships, some other relationships (e.g., item-item and user-user relationships) also greatly contribute to the effectiveness of GCN-based models on CF. However, in conventional GCN-based models, these relationships are implicitly learned through the same message passing layers with user-item relationships. This not only leads to the unreasonable edge weight assignments, but also fails to capture the relative importances of different types of relationships.

UltraGCN is flexible to extend to model many different relation graphs, such as user-user graphs, item-item graphs, and even knowledge graphs. For now, we will focus on the item-item co-occurrence graph, which has been shown to be useful for recommendation.

For each positive (𝑢, 𝑖) pair, we first construct 𝐾 weighted positive (𝑢, 𝑗) pairs, for 𝑗 ∈ 𝑆 (𝑖). Then, we penalize the learning of these pairs with the more reasonable similarity score $\omega_{𝑖,𝑗}$ and derive the constraint loss $\mathcal{L}_𝐼$ on the item-item graph as follow:

$$\mathcal{L}_I = \sum_{(u,i) \in N^+}\sum_{j \in S(i)} \omega_{i,j} \log (\sigma(e_u^Te_j))$$

We can omit the negative sampling here as the negative sampling in $\mathcal{L}_𝐶$ and $\mathcal{L}_O$ has already enabled UltraGCN to counteract over-smoothing. With this constraint loss, we can extend UltraGCN to better learn item-item relationships, and finally derive the following training objective of UltraGCN,

$$\mathcal{L} = \mathcal{L}_O + \lambda \mathcal{L}_C + \gamma\mathcal{L}_I$$

where 𝜆 and 𝛾 are hyper-parameters to adjust the relative importances of user-item and item-item relationships, respectively.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#export
class UltraGCN(nn.Module):
    def __init__(self, args, constraint_mat, ii_constraint_mat, ii_neighbor_mat):
        super(UltraGCN, self).__init__()
        self.user_num = args.user_num
        self.item_num = args.item_num
        self.embedding_dim = args.embedding_dim
        self.w1 = args.w1
        self.w2 = args.w2
        self.w3 = args.w3
        self.w4 = args.w4

        self.negative_weight = args.negative_weight
        self.gamma = args.gamma
        self.lambda_ = args.lambda_

        self.user_embeds = nn.Embedding(self.user_num, self.embedding_dim)
        self.item_embeds = nn.Embedding(self.item_num, self.embedding_dim)

        self.constraint_mat = constraint_mat
        self.ii_constraint_mat = ii_constraint_mat
        self.ii_neighbor_mat = ii_neighbor_mat

        self.initial_weight = args.initial_weight

        self.initial_weights()

    def initial_weights(self):
        nn.init.normal_(self.user_embeds.weight, std=self.initial_weight)
        nn.init.normal_(self.item_embeds.weight, std=self.initial_weight)

    def get_omegas(self, users, pos_items, neg_items):
        device = self.get_device()
        if self.w2 > 0:
            pos_weight = self.constraint_mat[users * self.item_num + pos_items].to(device)
            pow_weight = self.w1 + self.w2 * pos_weight
        else:
            pos_weight = self.w1 * torch.ones(len(pos_items)).to(device)
        
        users = (users * self.item_num).unsqueeze(0)

        if self.w4 > 0:
            neg_weight = self.constraint_mat[torch.cat([users] * neg_items.size(1)).transpose(1, 0) + neg_items].flatten().to(device)
            neg_weight = self.w3 + self.w4 * neg_weight
        else:
            neg_weight = self.w3 * torch.ones(neg_items.size(0) * neg_items.size(1)).to(device)

        weight = torch.cat((pow_weight, neg_weight))
        return weight

    def cal_loss_L(self, users, pos_items, neg_items, omega_weight):
        device = self.get_device()
        user_embeds = self.user_embeds(users)
        pos_embeds = self.item_embeds(pos_items)
        neg_embeds = self.item_embeds(neg_items)
      
        pos_scores = (user_embeds * pos_embeds).sum(dim=-1) # batch_size
        user_embeds = user_embeds.unsqueeze(1)
        neg_scores = (user_embeds * neg_embeds).sum(dim=-1) # batch_size * negative_num

        neg_labels = torch.zeros(neg_scores.size()).to(device)
        neg_loss = F.binary_cross_entropy_with_logits(neg_scores, neg_labels, weight = omega_weight[len(pos_scores):].view(neg_scores.size()), reduction='none').mean(dim = -1)
        
        pos_labels = torch.ones(pos_scores.size()).to(device)
        pos_loss = F.binary_cross_entropy_with_logits(pos_scores, pos_labels, weight = omega_weight[:len(pos_scores)], reduction='none')

        loss = pos_loss + neg_loss * self.negative_weight
      
        return loss.sum()

    def cal_loss_I(self, users, pos_items):
        device = self.get_device()
        neighbor_embeds = self.item_embeds(self.ii_neighbor_mat[pos_items].to(device))    # len(pos_items) * num_neighbors * dim
        sim_scores = self.ii_constraint_mat[pos_items].to(device)     # len(pos_items) * num_neighbors
        user_embeds = self.user_embeds(users).unsqueeze(1)
        
        loss = -sim_scores * (user_embeds * neighbor_embeds).sum(dim=-1).sigmoid().log()
      
        # loss = loss.sum(-1)
        return loss.sum()

    def norm_loss(self):
        loss = 0.0
        for parameter in self.parameters():
            loss += torch.sum(parameter ** 2)
        return loss / 2

    def forward(self, users, pos_items, neg_items):
        omega_weight = self.get_omegas(users, pos_items, neg_items)
        
        loss = self.cal_loss_L(users, pos_items, neg_items, omega_weight)
        loss += self.gamma * self.norm_loss()
        loss += self.lambda_ * self.cal_loss_I(users, pos_items)
        return loss

    def test_foward(self, users):
        items = torch.arange(self.item_num).to(users.device)
        user_embeds = self.user_embeds(users)
        item_embeds = self.item_embeds(items)
         
        return user_embeds.mm(item_embeds.t())

    def get_device(self):
        return self.user_embeds.weight.device

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-29 04:08:41

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

IPython: 5.5.0
torch  : 1.10.0+cu111

